# Классификация токсичных комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные со значением метрики качества *F1* не меньше 0.75. В распоряжении набор данных с разметкой о токсичности правок.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingClassifier
from catboost import Pool, cv, CatBoostClassifier
from tqdm.notebook import tqdm

In [2]:
data = pd.read_csv('./datasets/toxic_comments.csv')
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Все стоки заполнены, в данных нет пропусков

In [4]:
corpus = data['text'].astype('U')

Посмотрим на балансировку классов в выбоке.

In [5]:
data['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Выборка несбалансироварована по классам. Количество "токсичных" комментариев примерно в 9 раз меньше чем количество простых комментариев. Необходимо это учитывать при обучении моделей.

## Обучение

### Logistic Regression

In [6]:
stop_words = set(stopwords.words('english'))

In [7]:
lr_clf = make_pipeline(TfidfVectorizer(stop_words=stop_words), 
                       LogisticRegression(penalty='l2', solver='liblinear', C=11, 
                                          class_weight='balanced', random_state=42))

In [8]:
lr_cv_score = cross_val_score(lr_clf, corpus, data['toxic'], cv=5, scoring='f1')
lr_cv_score

array([0.77369495, 0.7682558 , 0.77161668, 0.77093668, 0.77205882])

In [9]:
'F1 Score {:.4f}'.format(lr_cv_score.mean())

'F1 Score 0.7713'

### Complement Naive Bayes

In [10]:
nb_clf = make_pipeline(TfidfVectorizer(stop_words=stop_words), 
                       ComplementNB(alpha=0.5))

In [11]:
nb_cv_score = cross_val_score(nb_clf, corpus, data['toxic'], cv=5, scoring='f1')
nb_cv_score

array([0.66780413, 0.65809943, 0.66564989, 0.65725944, 0.66380789])

In [12]:
'F1 Score {:.4f}'.format(nb_cv_score.mean())

'F1 Score 0.6625'

###  Passive Aggressive Classifier

In [13]:
pa_clf = make_pipeline(TfidfVectorizer(stop_words=stop_words), 
                       PassiveAggressiveClassifier(class_weight='balanced'))

In [14]:
pa_cv_score = cross_val_score(pa_clf, corpus, data['toxic'], cv=5, scoring='f1')
pa_cv_score

array([0.7336178 , 0.72481828, 0.73579109, 0.73796142, 0.73746762])

In [15]:
'F1 Score {:.4f}'.format(pa_cv_score.mean())

'F1 Score 0.7339'

### CatBoost

In [16]:
cv_pool = Pool(data=data[['text']], label=data[['toxic']], text_features=['text'])

In [17]:
params = {"iterations": 10,
          #"depth": 2,
          "loss_function": "Logloss",
          'eval_metric': 'F1',
          'custom_metric': 'F1',
          "scale_pos_weight": 9}

cb_scores = cv(cv_pool, params, fold_count=5, verbose=10, seed=42)

0:	learn: 0.6707687	test: 0.6707687	best: 0.6707687 (0)	total: 1.82s	remaining: 16.4s
9:	learn: 0.6707687	test: 0.6707687	best: 0.6707687 (0)	total: 15.1s	remaining: 0us


In [18]:
cb_scores

,iterations,test-F1-mean,test-F1-std,train-F1-mean,train-F1-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std,test-F1:use_weights=true-mean,test-F1:use_weights=true-std,train-F1:use_weights=true-mean,train-F1:use_weights=true-std,test-F1:use_weights=false-mean,test-F1:use_weights=false-std,train-F1:use_weights=false-mean,train-F1:use_weights=false-std
0,0,0.670769,0.000003,0.670769,8.612200e-07,0.693145,1.641390e-09,0.693145,1.066827e-09,0.670769,0.000003,0.670769,8.612200e-07,0.184589,0.000002,0.184589,5.869804e-07
1,1,0.670769,0.000003,0.670769,8.612200e-07,0.693142,3.264410e-09,0.693142,2.070704e-09,0.670769,0.000003,0.670769,8.612200e-07,0.184589,0.000002,0.184589,5.869804e-07
2,2,0.670769,0.000003,0.670769,8.612200e-07,0.693140,4.990489e-09,0.693140,2.892873e-09,0.670769,0.000003,0.670769,8.612200e-07,0.184589,0.000002,0.184589,5.869804e-07
3,3,0.670769,0.000003,0.670769,8.612200e-07,0.693138,6.566405e-09,0.693138,3.788969e-09,0.670769,0.000003,0.670769,8.612200e-07,0.184589,0.000002,0.184589,5.869804e-07
4,4,0.670769,0.000003,0.670769,8.612200e-07,0.693136,8.120855e-09,0.693136,4.631754e-09,0.670769,0.000003,0.670769,8.612200e-07,0.184589,0.000002,0.184589,5.869804e-07
5,5,0.670769,0.000003,0.670769,8.612200e-07,0.693134,9.653409e-09,0.693134,5.424947e-09,0.670769,0.000003,0.670769,8.612200e-07,0.184589,0.000002,0.184589,5.869804e-07
6,6,0.670769,0.000003,0.670769,8.612200e-07,0.693132,1.116296e-08,0.693132,6.171391e-09,0.670769,0.000003,0.670769,8.612200e-07,0.184589,0.000002,0.184589,5.869804e-07
7,7,0.670769,0.000003,0.670769,8.612200e-07,0.693131,1.264907e-08,0.693131,6.873640e-09,0.670769,0.000003,0.670769,8.612200e-07,0.184589,0.000002,0.184589,5.869804e-07
8,8,0.670769,0.000003,0.670769,8.612200e-07,0.693129,1.411082e-08,0.693129,7.534215e-09,0.670769,0.000003,0.670769,8.612200e-07,0.184589,0.000002,0.184589,5.869804e-07
9,9,0.670769,0.000003,0.670769,8.612200e-07,0.693128,1.554818e-08,0.693128,8.155974e-09,0.670769,0.000003,0.670769,8.612200e-07,0.184589,0.000002,0.184589,5.869804e-07


### Voting Classifier

In [19]:
# # ниже представлен код который переберает веса моделей и ищет наилучший голосующий классификатор по заданному критерию
# # так как перебор весов занимает порядка 45 мин код закомментирован.  
# # модель с уже найдеными весами представлена в следующей ячейке

# best_weights = []
# best_mean_score = 0

# for w0 in tqdm(np.arange(0, 1.1, 0.1)):
#     w1 = 1 - w0
#     voiting_clf = VotingClassifier(estimators=[('lr', lr_clf), ('nb', nb_clf)],
#                                   voting='soft', weights=[w0, w1])
#     voiting_clf_score = cross_val_score(voiting_clf, corpus, data['toxic'], cv=5, scoring='f1')
#     if voiting_clf_score.mean() > best_mean_score:
#         best_mean_score = voiting_clf_score.mean()
#         best_weights = [w0, w1]
# best_mean_score

In [20]:
voiting_clf = VotingClassifier(estimators=[('lr', lr_clf), ('nb', nb_clf)],
                              voting='soft', weights=[0.6, 0.4])

voiting_clf_score = cross_val_score(voiting_clf, corpus, data['toxic'], cv=5, scoring='f1')
voiting_clf_score

array([0.79107945, 0.78765432, 0.78515747, 0.78608964, 0.78873239])

In [21]:
'F1 Score {:.4f}'.format(voiting_clf_score.mean())

'F1 Score 0.7877'

## Сводная таблица

In [22]:
result = pd.DataFrame({'Model': ['Logistic Regression', 'Complement Naive Bayes', 'Passive Aggressive Classifier', 
                                 'CatBoost (Text Features)', 'Voting Classifier'],
                       'F1 Score Mean (5 Folds)': [lr_cv_score.mean(), nb_cv_score.mean(), pa_cv_score.mean(), 
                                            cb_scores.iloc[-1, 1], voiting_clf_score.mean()],
                       'F1 Score Std (5 Folds)': [lr_cv_score.std(), nb_cv_score.std(), pa_cv_score.std(), 
                                            cb_scores.iloc[-1, 2], voiting_clf_score.std()]
                     })


(result.set_index('Model')
       .sort_values('F1 Score Mean (5 Folds)', ascending=False)
       .style.format({'F1 Score Mean (5 Folds)': "{:.4%}", 'F1 Score Std (5 Folds)': '{:.4%}'}))

,F1 Score Mean (5 Folds),F1 Score Std (5 Folds)
Model,,
Voting Classifier,78.7743%,0.2075%
Logistic Regression,77.1313%,0.1778%
Passive Aggressive Classifier,73.3931%,0.4803%
CatBoost (Text Features),67.0769%,0.0003%
Complement Naive Bayes,66.2524%,0.4162%


По значению метрики F1 наилучшей является модель Voting Classifier.

## Выводы

В процессе исследования были рассмотренны следующие модели:
* Logistic Regression;
* Complement Naive Bayes;
* Passive Aggressive Classifier;
* CatBoost;
* Voting Classifier (Logistic Regression + Complement Naive Bayes).

Из рассмотренных моделей заданным требованиям (метрика качества *F1* не меньше 0.75) соответствуют только модели Logistic Regression и Voting Classifier. Модель логистической регрессии дает метрику F1 - 0.7713. Объединение модели логистической регрессии и байесовской модели дает улучшение метрики до 0,7877.